In [77]:
print("Hello, I am learning cash flow modeling")



Hello, I am learning cash flow modeling


In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [79]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [80]:
import os
os.listdir()


['.config', 'drive', 'ALL_SAMPLE_DATASETS (1).xlsx', 'sample_data']

In [81]:
df = pd.read_excel('ALL_SAMPLE_DATASETS (1).xlsx')

In [82]:
xls = pd.ExcelFile('ALL_SAMPLE_DATASETS (1).xlsx')
xls.sheet_names


['healthy_business_90days',
 'struggling_business_90days',
 'variable_business_90days',
 'healthy_business_10days',
 'healthy_business_30days',
 'healthy_business_60days']

In [83]:
df = pd.read_excel(
    'ALL_SAMPLE_DATASETS (1).xlsx',
    sheet_name= 'healthy_business_90days'
)

df.head()


,current_balance,transaction_id,date,type,amount,category,description
0,800000,txn_0001,2025-11-11T08:51:46Z,income,37227,takeout,Day 1 sales
1,800000,txn_0002,2025-11-11T08:51:46Z,income,32478,daily_sales,Day 1 sales
2,800000,txn_0003,2025-11-11T08:51:46Z,expense,500,spices,Day 1 expenses
3,800000,txn_0004,2025-11-11T08:51:46Z,expense,6055,meat,Day 1 expenses
4,800000,txn_0005,2025-11-11T08:51:46Z,expense,2780,rent,Day 1 expenses


In [84]:
df.columns


Index(['current_balance', 'transaction_id', 'date', 'type', 'amount',
       'category', 'description'],
      dtype='object')

In [85]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   current_balance  576 non-null    int64 
 1   transaction_id   576 non-null    object
 2   date             576 non-null    object
 3   type             576 non-null    object
 4   amount           576 non-null    int64 
 5   category         576 non-null    object
 6   description      576 non-null    object
dtypes: int64(2), object(5)
memory usage: 31.6+ KB


In [86]:
df.describe()


,current_balance,amount
count,576.0,576.000000
mean,800000.0,19457.789931
std,0.0,13244.221547
min,800000.0,500.000000
25%,800000.0,9294.500000
50%,800000.0,16290.500000
75%,800000.0,27639.250000
max,800000.0,72431.000000


In [87]:
#ENSURING THAT THE DATA IS IN THE RIGHT FORMAT
df['date'] = pd.to_datetime(df['date'])


In [88]:
# creating year-month column
df['year_month'] = df['date'].dt.to_period('M')


/tmp/ipython-input-2509827035.py:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['year_month'] = df['date'].dt.to_period('M')


In [89]:
# Seperating income from expenses
df['type'].unique()


array(['income', 'expense'], dtype=object)

In [90]:
monthly = df.groupby(['year_month', 'type'])['amount'].sum().unstack()
monthly


type,expense,income
year_month,,
2025-11,661024,1851624
2025-12,999748,2942521
2026-01,1017542,2741476
2026-02,258025,735727


In [91]:
#CREATING MONTHLY CASHFLOW SUMMARY

# Group by month and type, sum amounts
monthly = df.groupby(['year_month', 'type'])['amount'].sum().unstack().fillna(0)

# Create total expenses column (already labeled 'expense') and net cash flow
monthly['net_cashflow'] = monthly['income'] - monthly['expense']

# Reset index to make it easy to work with
monthly = monthly.reset_index()

monthly.head()


type,year_month,expense,income,net_cashflow
0,2025-11,661024,1851624,1190600
1,2025-12,999748,2942521,1942773
2,2026-01,1017542,2741476,1723934
3,2026-02,258025,735727,477702


In [92]:
#Adding negative cashflow flag
monthly['negative_cashflow'] = (monthly['net_cashflow'] < 0).astype(int)
monthly.head()


type,year_month,expense,income,net_cashflow,negative_cashflow
0,2025-11,661024,1851624,1190600,0
1,2025-12,999748,2942521,1942773,0
2,2026-01,1017542,2741476,1723934,0
3,2026-02,258025,735727,477702,0


In [93]:
#LAG FEATURES (past months cash flow)
# Previous month cash flow
monthly['lag_1'] = monthly['net_cashflow'].shift(1)

# 3-month lag
monthly['lag_3'] = monthly['net_cashflow'].shift(3)

# Rolling 3-month average
monthly['rolling_3_mean'] = monthly['net_cashflow'].rolling(3).mean()


In [94]:
#Ratios
monthly['expense_ratio'] = monthly['expense'] / (monthly['income'] + 1e-6)  # prevent divide by zero


In [95]:
#Time features
monthly['month'] = monthly['year_month'].dt.month
monthly['quarter'] = monthly['year_month'].dt.quarter


In [96]:
#few first roles wtih NaN from Lags
monthly = monthly.dropna().reset_index(drop=True)
monthly.head()


type,year_month,expense,income,net_cashflow,negative_cashflow,lag_1,lag_3,rolling_3_mean,expense_ratio,month,quarter
0,2026-02,258025,735727,477702,0,1723934.0,1190600.0,1.381470e+06,0.350708,2,1


Spliting data for machine leraning

In [97]:
#preparing Features $ Tagrets
features = ['income', 'expense', 'lag_1', 'lag_3', 'rolling_3_mean', 'expense_ratio', 'month', 'quarter']

# Regression target
X = monthly[features]
y_reg = monthly['net_cashflow']

# Classification target
y_clf = monthly['negative_cashflow']


In [98]:
# Time- based train/ test spilit
train_size = int(len(monthly) * 0.8)

X_train = X.iloc[:train_size]
X_test = X.iloc[train_size:]

y_reg_train = y_reg.iloc[:train_size]
y_reg_test = y_reg.iloc[train_size:]

y_clf_train = y_clf.iloc[:train_size]
y_clf_test = y_clf.iloc[train_size:]


In [99]:
# Regression to Predict Net_flow
features = [
    'income',
    'expense',
    'lag_1',
    'lag_3',
    'rolling_3_mean',
    'expense_ratio',
    'month',
    'quarter'
]


In [100]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd

In [101]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error


In [102]:
rf_model = RandomForestRegressor(
    n_estimators=200,
    random_state=42
)


In [103]:
rf_model


RandomForestRegressor(n_estimators=200, random_state=42)

In [104]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error


In [105]:
df = pd.read_excel ("ALL_SAMPLE_DATASETS (1).xlsx")
df.head()


,current_balance,transaction_id,date,type,amount,category,description
0,800000,txn_0001,2025-11-11T08:51:46Z,income,37227,takeout,Day 1 sales
1,800000,txn_0002,2025-11-11T08:51:46Z,income,32478,daily_sales,Day 1 sales
2,800000,txn_0003,2025-11-11T08:51:46Z,expense,500,spices,Day 1 expenses
3,800000,txn_0004,2025-11-11T08:51:46Z,expense,6055,meat,Day 1 expenses
4,800000,txn_0005,2025-11-11T08:51:46Z,expense,2780,rent,Day 1 expenses


In [106]:
features = [
    'income',
    'expense',
    'lag_1',
    'lag_3',
    'rolling_3_mean',
    'expense_ratio',
    'month',
    'quarter'
]

X = monthly[features]
y = monthly['net_cashflow']


In [107]:
monthly.head()


type,year_month,expense,income,net_cashflow,negative_cashflow,lag_1,lag_3,rolling_3_mean,expense_ratio,month,quarter
0,2026-02,258025,735727,477702,0,1723934.0,1190600.0,1.381470e+06,0.350708,2,1


In [108]:
train_size = int(len(monthly) * 0.8)

X_train = X.iloc[:train_size]
X_test = X.iloc[train_size:]

y_train = y.iloc[:train_size]
y_test = y.iloc[train_size:]


In [109]:
monthly.shape


(1, 11)

In [110]:
# Rebuild features safely
monthly = monthly.reset_index(drop=True)

monthly['lag_1'] = monthly['net_cashflow'].shift(1)
monthly['expense_ratio'] = monthly['expense'] / (monthly['income'] + 1e-6)
monthly['month'] = monthly['year_month'].dt.month
monthly['quarter'] = monthly['year_month'].dt.quarter

# Drop only 1 row
monthly = monthly.dropna().reset_index(drop=True)


In [111]:
monthly.shape
monthly.head()


type,year_month,expense,income,net_cashflow,negative_cashflow,lag_1,lag_3,rolling_3_mean,expense_ratio,month,quarter


In [112]:
features = [
    'income',
    'expense',
    'lag_1',
    'expense_ratio',
    'month',
    'quarter'
]

X = monthly[features]
y = monthly['net_cashflow']


In [113]:
train_size = int(len(monthly) * 0.8)

X_train = X.iloc[:train_size]
X_test = X.iloc[train_size:]

y_train = y.iloc[:train_size]
y_test = y.iloc[train_size:]


In [114]:
monthly = monthly.reset_index(drop=True)

monthly['expense_ratio'] = monthly['expense'] / (monthly['income'] + 1e-6)
monthly['month'] = monthly['year_month'].dt.month
monthly['quarter'] = monthly['year_month'].dt.quarter


In [115]:
features = ['income', 'expense', 'expense_ratio', 'month', 'quarter']

X = monthly[features]
y = monthly['net_cashflow']


In [116]:
# Reconstruct monthly DataFrame to ensure it's not empty
# These steps replicate the necessary data preparation from previous cells
# to ensure 'monthly' is correctly populated for model training.

# Ensure 'date' column is datetime and create 'year_month'
# (assuming df is available from previous cells)
import pandas as pd

if 'date' not in df.columns or not pd.api.types.is_datetime64_any_dtype(df['date']):
    df['date'] = pd.to_datetime(df['date'])
if 'year_month' not in df.columns:
    df['year_month'] = df['date'].dt.to_period('M')

# Group by month and type, sum amounts
monthly = df.groupby(['year_month', 'type'])['amount'].sum().unstack().fillna(0)

# Create net cash flow
monthly['net_cashflow'] = monthly['income'] - monthly['expense']

# Reset index to make it easy to work with
monthly = monthly.reset_index()

# Calculate expense ratio, month, and quarter
monthly['expense_ratio'] = monthly['expense'] / (monthly['income'] + 1e-6)
monthly['month'] = monthly['year_month'].dt.month
monthly['quarter'] = monthly['year_month'].dt.quarter

# Define features and target. Exclude lag features if the dataset is too small
# as they caused most of the NaNs leading to empty dataframe.
features = ['income', 'expense', 'expense_ratio', 'month', 'quarter']
X = monthly[features]
y = monthly['net_cashflow']

# Initialize and fit the model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X, y)

/tmp/ipython-input-2712153476.py:12: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['year_month'] = df['date'].dt.to_period('M')


RandomForestRegressor(random_state=42)

Cahflow Analysis

In [117]:
#Cashfow matrix
monthly['net_cashflow']


,net_cashflow
0,1190600
1,1942773
2,1723934
3,477702


In [118]:
cashflow_volatility = monthly['net_cashflow'].std()
cashflow_volatility


652294.0720178157

In [119]:
monthly['income_expense_ratio'] = monthly['income'] / (monthly['expense'] + 1e-6)
monthly[['year_month', 'income_expense_ratio']]


type,year_month,income_expense_ratio
0,2025-11,2.801145
1,2025-12,2.943263
2,2026-01,2.694214
3,2026-02,2.851379


In [120]:
# identifying risky months
risky_months = monthly[monthly['net_cashflow'] < 0]
risky_months[['year_month', 'net_cashflow']]


type,year_month,net_cashflow


In [121]:
expense_breakdown = (
    df[df['type'] == 'expense']
    .groupby('category')['amount']
    .sum()
    .sort_values(ascending=False)
)

expense_breakdown


,amount
category,
meat,526187
vegetables,384660
raw_materials,383665
utilities,370449
spices,362483
salaries,351079
transportation,303417
rent,254399


In [122]:
end_of_month_balance = (
    df.sort_values('date')
      .groupby('year_month')['current_balance']
      .last()
)

end_of_month_balance


,current_balance
year_month,
2025-11,800000
2025-12,800000
2026-01,800000
2026-02,800000


In [123]:
#income source anaylsis
income_sources = (
    df[df['type'] == 'income']
    .groupby('category')['amount']
    .sum()
    .sort_values(ascending=False)
)

income_sources


,amount
category,
takeout,2406152
daily_sales,2366583
delivery,1985159
catering,1513454


In [124]:
# Cah balance health
end_of_month_balance = (
    df.sort_values('date')
      .groupby('year_month')['current_balance']
      .last()
)

end_of_month_balance


,current_balance
year_month,
2025-11,800000
2025-12,800000
2026-01,800000
2026-02,800000


In [125]:
#risk flags
monthly['risk_flag'] = (
    (monthly['net_cashflow'] < 0) |
    (monthly['income_expense_ratio'] < 1)
).astype(int)

monthly[['year_month', 'net_cashflow', 'income_expense_ratio', 'risk_flag']]


type,year_month,net_cashflow,income_expense_ratio,risk_flag
0,2025-11,1190600,2.801145,0
1,2025-12,1942773,2.943263,0
2,2026-01,1723934,2.694214,0
3,2026-02,477702,2.851379,0
